# Mask R-CNN - Inspect RPN Model

Code and visualizations to test, debug, and evaluate the RPN model.

In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Root directory of the project
ROOT_DIR = "C:\\Users\\Martin\\Documents\\LEGOFinder\\Keras\\lego_object_detection\\maskrcnn"

%load_ext autoreload
%autoreload 2

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

from samples.lego import lego

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")


In [ ]:
# Path to Lego trained weights
#LEGO_WEIGHTS_PATH = "C:/Users/Martin/Documents/LEGOFinder/Keras/lego_object_detection/maskrcnn/snapshots/mask_rcnn_lego_0025.h5" # Referenz Gewichte mit guten Resultaten
#LEGO_WEIGHTS_PATH = "C:/Users/Martin/Google Drive/Colab/maskrcnn/snapshots/lego20200614T1246/mask_rcnn_lego_0020.h5"
LEGO_WEIGHTS_PATH = "C:/Users/Martin/Documents/LEGOFinder/Keras/lego_object_detection/maskrcnn/snapshots/lego20200620T1156/mask_rcnn_lego_0040.h5"

# load either "train", "val" or "eval"
DATASET = "eval"

## Configurations

In [ ]:
config = lego.LegoConfig()
LEGO_DIR = os.path.join(ROOT_DIR, "datasets", "lego")

In [ ]:
# Override the training configurations with a few
# changes for inferencing.
class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0.8
    USE_LRPN = False

config = InferenceConfig()
config.display()

## Notebook Preferences

In [ ]:
# Inspect the model in training or inference modes
# values: 'inference' or 'training'
# TODO: code for 'training' test mode not ready yet
TEST_MODE = "inference"

In [ ]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

## Load Dataset

In [ ]:
# Load validation dataset
dataset = lego.LegoDataset()
dataset.load_lego(LEGO_DIR, DATASET)

# Must call before using the dataset
dataset.prepare()

print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

## Load Model

In [ ]:
# Create model in inference mode
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

In [ ]:
# Set path to lego weights file

# Load weights
print("Loading weights ", LEGO_WEIGHTS_PATH)
model.load_weights(LEGO_WEIGHTS_PATH, by_name=True)

## Stage 1: Region Proposal Network

The Region Proposal Network (RPN) runs a lightweight binary classifier on a lot of boxes (anchors) over the image and returns object/no-object scores. Anchors with high *objectness* score (positive anchors) are passed to the stage two to be classified.

Often, even positive anchors don't cover objects fully. So the RPN also regresses a refinement (a delta in location and size) to be applied to the anchors to shift it and resize it a bit to the correct boundaries of the object.

In [ ]:

# Choose image to analyse
if DATASET == "eval":
    image_id = dataset.get_image_id("0000000002.png") # if eval set us the image with the Lego haufen
else:
    image_ids = np.random.choice(dataset.image_ids, 1)
    image_id = image_ids[0]
    
image, image_meta, gt_class_ids, gt_bboxes, gt_masks = modellib.load_image_gt(dataset, config, image_id, use_mini_mask=False)

# select some GT boxes
idx = [4, 8]

#gt_bboxes = np.array(gt_bboxes[idx])
#gt_class_ids = np.array(gt_class_ids[idx])



### 1.a RPN Targets

The RPN targets are the training values for the RPN. To generate the targets, we start with a grid of anchors that cover the full image at different scales, and then we compute the IoU of the anchors with ground truth object. Positive anchors are those that have an IoU >= 0.7 with any ground truth object, and negative anchors are those that don't cover any object by more than 0.3 IoU. Anchors in between (i.e. cover an object by IoU >= 0.3 but < 0.7) are considered neutral and excluded from training.

To train the RPN regressor, we also compute the shift and resizing needed to make the anchor cover the ground truth object completely.
 

In [ ]:
# Generate RPN trainig targets (input for RPN)
if config.USE_RPN_ROIS:

    # Call get_anchors() to load anchors
    molded_images, image_metas, windows = model.mold_inputs([image])
    image_shape = molded_images[0].shape
    model.get_anchors(image_shape)

    target_rpn_match, target_rpn_bbox = modellib.build_rpn_targets( image.shape, model.anchors, gt_class_ids, gt_bboxes, model.config)

    log("target_rpn_match", target_rpn_match)       # input_rpn_match -> für jeden Anchor ob dieser ein     1 = positive anchor (IoU > 0.7), 
                                                    #                                                      -1 = negative anchor (IoU < 0.3), 
                                                    #                                                       0 = neutral is im Vergleich zu den GT boxen
    log("target_rpn_bbox", target_rpn_bbox)         # input_rpn_bbox ->  Anchor bbox deltas against the GT boxes. This is the delta the RPN shout predict.

    positive_anchor_ix = np.where(target_rpn_match[:] == 1)[0]
    negative_anchor_ix = np.where(target_rpn_match[:] == -1)[0]
    neutral_anchor_ix = np.where(target_rpn_match[:] == 0)[0]
    positive_anchors = model.anchors[positive_anchor_ix]
    negative_anchors = model.anchors[negative_anchor_ix]
    neutral_anchors = model.anchors[neutral_anchor_ix]
    log("positive_anchors", positive_anchors)
    log("negative_anchors", negative_anchors)
    log("neutral anchors", neutral_anchors)

    # Apply refinement deltas to positive anchors
    refined_anchors = utils.apply_box_deltas(
        positive_anchors,
        target_rpn_bbox[:positive_anchors.shape[0]] * model.config.RPN_BBOX_STD_DEV)
    log("refined_anchors", refined_anchors, )

    # Print dimensions of positive anchors
    print("\nBest matching anchors are (Image Dimensions={}):\n".format(image.shape))
    for i, rect in enumerate(positive_anchors):
        y1, x1, y2, x2 = rect
        
        print("Anchor {}: \t\t [{:.0f} {:.0f} {:.0f} {:.0f}] \tw={:.0f} \th={:.0f}".format(i, x1, y1, x2, y2, x2-x1, y2-y1))


In [ ]:
# ACHTUNG DAS IST DER IDEAL FALL !!! 
#
# Das Bild unten zeigt nur die refinded Boxen basieren auf GT den Boxen, nicht die vom RPN vorgeschlagenen (Proposals), siehe 
# nächste Sektion unten. Es ist der Best Case, wenn das RPN perfekt funktionieren würde.
#

if config.USE_RPN_ROIS:
    visualize.draw_boxes(image, boxes=positive_anchors, refined_boxes=refined_anchors, ax=get_ax())

### 1.b RPN Predictions

Here we run the RPN graph and display its predictions.

In [ ]:
# Run RPN sub-graph
if config.USE_RPN_ROIS:
    pillar = model.keras_model.get_layer("ROI").output

    # TF 1.4 and 1.9 introduce new versions of NMS. Search for all names to support TF 1.3~1.10
    nms_node = model.ancestor(pillar, "ROI/rpn_non_max_suppression:0")
    if nms_node is None:
        nms_node = model.ancestor(pillar, "ROI/rpn_non_max_suppression/NonMaxSuppressionV2:0")
    if nms_node is None: #TF 1.9-1.10
        nms_node = model.ancestor(pillar, "ROI/rpn_non_max_suppression/NonMaxSuppressionV3:0")

    rpn = model.run_graph([image], [
        ("rpn_class", model.keras_model.get_layer("rpn_class").output),         # Letzer Layer im RPN mit den propsosed Klassen Wahrscheinlichkeit GB oder FG pro Anker
        ("pre_nms_anchors", model.ancestor(pillar, "ROI/pre_nms_anchors:0")),
        ("refined_anchors", model.ancestor(pillar, "ROI/refined_anchors:0")),
        ("refined_anchors_clipped", model.ancestor(pillar, "ROI/refined_anchors_clipped:0")),
        ("post_nms_anchor_ix", nms_node),                                       # Filtered anchors that have a NMS of RPN_NMS_THRESHOLD (>0.7)
        ("proposals", model.keras_model.get_layer("ROI").output),               # "ROI" ist name des ProposalLayers(), also die gefilterten rois proposals
    ], config)

### 1.c Analyse RPN Proposal Accuracy

The idea is here to check how well the proposals match the GT. Hence for every lego (GT box) we check what is the maximum IoU. Then we simply form the average to get a single value for its performance.

In [ ]:
# Find the proposal that has the highest IoU on each GT box and take average of all
max_proposals_bbox = np.empty((gt_bboxes.shape[0], 4))
max_proposals_iou = np.empty((gt_bboxes.shape[0], 1))

# Get proposals from graph
h, w = config.IMAGE_SHAPE[:2]
proposals = rpn['proposals'][0, :] * np.array([h, w, h, w])

print("Best proposals are:\n")
for i, box in enumerate(gt_bboxes):

    overlaps = utils.compute_overlaps(np.array([box.tolist()]), proposals)
    iou_max = np.max(overlaps, axis=1)
    iou_argmax = np.argmax(overlaps, axis=1)
    max_proposals_bbox[i] = proposals[iou_argmax]
    max_proposals_iou[i] = iou_max

    y1, x1, y2, x2 = proposals[iou_argmax][0]
    iou = iou_max[0]

    print("Proposal Bbox {}: \t[{:.0f} {:.0f} {:.0f} {:.0f}] w={:.0f} h={:.0f} iou={:.2f}".format(i, x1, y1, x2, y2, x2-x1, y2-y1, iou))


print("\nAverage maximum IoU is: {0:0.2f}".format(np.average(max_proposals_iou)))
visualize.draw_boxes(image, boxes=gt_bboxes, refined_boxes=max_proposals_bbox)

### 1.d Analyse RPN Anchors for GT Boxes in Detail

Here we try to find the anchors with the highest probability for a certain GT. The question is if the RPN has predicted the right anchors.

In [ ]:
if config.USE_RPN_ROIS:

    # Get anchor probabilties, second column with FG probabilites of an anchor  
    anchors_prob_fg = rpn['rpn_class'][:,:,1].flatten()

    # For each GT box, find anchors which overlap at least a little bit and get the propbability that the RPN has predicted for this anchor
    for i, box in enumerate(gt_bboxes):

        print("\n--> Analysing GT box {}\n".format(i))
        matched_anchors = []
        overlaps = utils.compute_overlaps(model.anchors, np.array([box.tolist()]))

        # Get anchor indices where IoU > x
        anchor_idxs = np.where(overlaps >= 0.2)[0]

        for i,idx in enumerate(anchor_idxs):

            # use only anchors with a minimum probability only
            if anchors_prob_fg[idx] >= 0.9:

                print("    Anchor {:.0f} IoU={:0.2f}, probability={:0.4f}".format(idx, overlaps[idx][0], anchors_prob_fg[idx]))
                matched_anchors.append(model.anchors[idx])

        if len(matched_anchors) == 0:
            print("Ups - no anchor found with this IoU and probability. Adjust values.")
        else:
            visualize.draw_boxes(image, boxes=np.asarray(matched_anchors), ax=get_ax())

In [ ]:
# Show top anchors by score (before refinement) -> here every lego should have at least one anchor, otherwise something is wrong.
if config.USE_RPN_ROIS:
    limit = 100
    # Take second column with FG probabilites
    anchors_prob_fg = rpn['rpn_class'][:,:,1].flatten()
    sorted_anchor_ids = np.argsort(anchors_prob_fg)[::-1]
    print("Top 100 anchor probabilties, max. {0:0.2f}, min. {1:0.2f}".format(
            anchors_prob_fg[sorted_anchor_ids[0]], anchors_prob_fg[sorted_anchor_ids[limit-1]]))
    visualize.draw_boxes(image, boxes=model.anchors[sorted_anchor_ids[:limit]], ax=get_ax())

In [ ]:
# Show top anchors with refinement. Then with clipping to image boundaries
if config.USE_RPN_ROIS:
    limit = 200
    ax = get_ax(1, 2)
    pre_nms_anchors = utils.denorm_boxes(rpn["pre_nms_anchors"][0], image.shape[:2])
    refined_anchors = utils.denorm_boxes(rpn["refined_anchors"][0], image.shape[:2])
    refined_anchors_clipped = utils.denorm_boxes(rpn["refined_anchors_clipped"][0], image.shape[:2]) # vor den NMS
    visualize.draw_boxes(image,     boxes           = pre_nms_anchors[:limit],
                                    refined_boxes   = refined_anchors[:limit], ax=ax[0])
    visualize.draw_boxes(image,     refined_boxes   = refined_anchors_clipped[:limit], ax=ax[1])

In [ ]:
# Show refined anchors after non-max suppression. -> Here some anchors from previous pictures might fall away that have overlays with other, to give place for new ones. This is a bit confusing to look at, since on the previous image, we might have seen less anchors.
if config.USE_RPN_ROIS:
    limit = 50
    ixs = rpn["post_nms_anchor_ix"][:limit]
    visualize.draw_boxes(image, refined_boxes=refined_anchors_clipped[ixs], ax=get_ax())

In [ ]:
# Show final proposals.
if config.USE_RPN_ROIS:
    limit = 20
    # Convert back to image coordinates for display
    h, w = config.IMAGE_SHAPE[:2]
    proposals = rpn['proposals'][0, :limit] * np.array([h, w, h, w])
    visualize.draw_boxes(image, refined_boxes=proposals, ax=get_ax())

MW: Final rois proposals from RPN network. Idealerweise sollten diese sehr ähnlich den GT Boxen sein.